In [ ]:
from load_image import *


# Inference using T2I

In [ ]:
from T2I import *

In [ ]:
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

# replace prompt as required by reference image

prompt=["Beautiful, carpet, 4k picture, high quality, color gradient rainbow , make it dim , similar to input image",
       "pattern,orange,red,yellow,purple,blue,artistic,floral,lace,motifs",
        "design,abstract,rich,dynamic,swirls,deep,vivid,decorative,warm,complex",
        "intricate,colorful,ornate,paisley,floral,detailed,swirling,vibrant,textured,elegant",
        "motifs,artistic,pattern,deep,warm,vivid,intricate,ornate,colorful,dynamic"]


In [ ]:
steps=5 # keep steps same as number of prompts and the image you want to generate
for i in range(steps):
  gen_images = pipe(
    prompt=prompt[i],
    negative_prompt=negative_prompt,
    image=image,
    num_inference_steps=30,
    guidance_scale=7,
    adapter_conditioning_scale=0.9,
    adapter_conditioning_factor=1
  ).images[0]
  gen_images.save(f'images/out_carpet{i}.png')

# inference using IP adapter


In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image
import torch

pipeline = AutoPipelineForImage2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")
pipeline.set_ip_adapter_scale(0.6)

In [ ]:
image = load_image("https://designcompetition.explorug.online/images/Border/2/16.webp")
ip_image = load_image("https://designcompetition.explorug.online/images/Border/2/17.webp")

generator = torch.Generator(device="cpu").manual_seed(4)
images_l=[]
for _ in range(5):
    images = pipeline(
        prompt="best quality, high quality",
        image=image,
        ip_adapter_image=ip_image,
        generator=generator,
        strength=0.6,
    ).images[0]
    images_l.append(images)



# infer using stable_diffusion with clip
CLIP interogator generates prompts for the input/reference image , that prompt is passed through runway/stable_diffusion to generate new images that resembles original image

In [ ]:
from CLIP import *
from stability import *

In [ ]:
folder_path = "/content" #@param {type:"string"}
prompt_mode = 'best' #@param ["best","fast","classic","negative"]
output_mode = 'desc.csv' #@param ["desc.csv","rename"]
max_filename_len = 128 #@param {type:"integer"}


In [ ]:
from clip_interrogator import Config, Interrogator

caption_model_name = 'blip-large' #@param ["blip-base", "blip-large", "git-large-coco"]
clip_model_name = 'ViT-L-14/openai' #@param ["ViT-L-14/openai", "ViT-H-14/laion2b_s32b_b79k"]

config = Config()
config.clip_model_name = clip_model_name
config.caption_model_name = caption_model_name
ci = Interrogator(config)

ci.config.quiet = True


In [ ]:
generate_prompt(folder_path,prompt_mode,output_mode,max_filename_len)